# Travelplan Optimization using MIS Demo
---
This notebook uses [pyTigerGraph](https://pytigergraph.github.io/pyTigerGraph/), a TigerGraph python interface to run gsql queries on a remote server running TigerGraph via Rest APIs.


### Setup
---
Boilerplate module imports

In [1]:
import time
import random as rand
from pathlib import Path, PurePosixPath
import pyTigerGraph as tg

#### Login Setup
Provide the remote TigerGraph server URL/IP address/hostname and credentials for a TigerGraph user. 

**NOTE**: The TigerGraph user should be created on the server side before proceeding

In [2]:
hostName = "localhost"                              # TG server hostname
userName = "tigergraph"                             # TG user name
passWord = "Xilinx123"                             # TG user password

#### Path Setup
**Local**: Location of query files under the Xilinx graphanalytics github repo. Set location of the local repo.

In [3]:
localRepoLocation = Path("/opt/xilinx/apps")
exampleLocation = Path("graphanalytics/integration/Tigergraph-3.x/mis/0.2/examples/travelplan/") # when running from github repo
queryFileLocation = localRepoLocation / exampleLocation / "query"

**NOTE**: Data should exist on the TigerGraph server 

In [11]:
serverRepoLocation = PurePosixPath("/opt/xilinx/apps")
serverDataLocation = serverRepoLocation / PurePosixPath(exampleLocation) / "data"
tp2woInfile = serverDataLocation / "travelplan2workorders100.csv"
tp2trInfile = serverDataLocation / "travelplan2trucks100.csv"

---
#### Prepare TG database
Shows **one-time** preparation of the database. Once done, queries can be repeateadly run as shown in the next Section.
1. [**Load Graph**](#loadg)
 - [Create new graph](#newg)
 - [Create graph schema](#schema)
 - [Load graph data](#loadd)
 - [Install queries](#install)


2. [**Build Edges**](#build_edges)


#### Run Queries on FPGA
Shows **repeatable** use of query to run *accelerated* MIS on FPGA
1. [**Run MIS**](#run)

The cells below show how to perform these steps in detail.

### 1. Load Graph <a id="loadg"></a>
---
#### 1.1 Create new graph <a id="newg"></a>
- Connect to TigerGraph server by ommiting graph name. This is needed to establish a valid REST endpoint that will be used to create a new desired graph
- Create new graph by using gsql command and create a new connection with the new graph

In [5]:
# connect to TG server and create graph
graphName = f'xgraph_{userName}'   # TG graph name
conn = tg.TigerGraphConnection(host='http://' + hostName, graphname='', username=userName, password=passWord, useCert=False)
print("\n--------- Creating New graph ----------")
print(conn.gsql(f'create graph {graphName}()', options=[]))

# connect to TG server with new graph
print(f'Using graph {graphName}')
conn = tg.TigerGraphConnection(host='http://' + hostName, graphname=graphName, username=userName, password=passWord, useCert=False)


--------- Creating New graph ----------
The graph xgraph_tigergraph is created.
Using graph xgraph_tigergraph


Any command or query will now run on the new graph.

#### 1.2 Create graph schema <a id="schema"></a>
TigerGraph stores graph in the form of vertices that can be associated with other vertices using directed or undirected edges. This is specified in the form of a graph schema. For the purpose of this demo, the schema is already defined as a query file. Load the file, set graph name and run it as gsql commands. 

The user can create schema for their own graph in a similar way. 

In [6]:
print("\n--------- Creating New Schema ----------")
schemaFile = queryFileLocation / "schema.gsql"

with open(schemaFile) as fh:
    qStrRaw = fh.read()
    qStr = qStrRaw.replace('@graph', graphName)
    print(conn.gsql(qStr))


--------- Creating New Schema ----------
Using graph 'xgraph_tigergraph'
The graph xgraph_tigergraph is dropped.
The graph xgraph_tigergraph is created.
The job job_schema_change_local is created.

Current graph version 0
Trying to add vertex travel_plan.
Trying to add vertex work_order.
Trying to add vertex truck.
Trying to add edge tp2wo.
Trying to add edge tp2truck.
Trying to add edge tp2tp.
Kick off job job_schema_change_local

Graph xgraph_tigergraph update to new version 1
The job job_schema_change_local completes in 10.632 seconds!
The job job_schema_change_local is dropped!


#### 1.3 Load graph data <a id="loadd"></a>

In [24]:
print("\n--------- Loading data into graph ----------")
loadFile = queryFileLocation / "load.gsql"

with open(loadFile) as fh:
    qStrRaw = fh.read()
    qStr = qStrRaw.replace('@graph', graphName)
    print(conn.gsql(qStr))
    print(conn.gsql(f'USE GRAPH {graphName}\n RUN LOADING JOB load_xgraph USING tp2wo_infile="{tp2woInfile}", tp2tr_infile="{tp2trInfile}"'))
    print(conn.gsql(f"USE GRAPH {graphName}\n DROP JOB load_xgraph"))


--------- Loading data into graph ----------
Semantic Check Fails: The USING clause for the same file path "null" should be the same. However in Job 'load_xgraph' one block has USING clause as "{SEPARATOR=,, USER_DEFINED_HEADER=tp2tr_header}", while another block has USING clause as "{SEPARATOR=,, USER_DEFINED_HEADER=tp2wo_header}".
Using graph 'xgraph_tigergraph'
The job load_xgraph is created.



Using graph 'xgraph_tigergraph'
[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_xgraph, jobid: xgraph_tigergraph.load_xgraph.file.m1.1643956970716
Loading log: '/home2/tigergraph/tigergraph/log/restpp/restpp_loader_logs/xgraph_tigergraph/xgraph_tigergraph.load_xgraph.file.m1.1643956970716.log'

Job "xgraph_tigergraph.load_xgraph.file.m1.1643956970716" loading status
[RUNNING] m1 ( Finished

#### 1.4 Install queries <a id="install"></a>

In [26]:
print("\n--------- Installing Queries ----------")
queryFiles = [queryFileLocation / "build_edges.gsql",
              queryFileLocation / "tg_maximal_indep_set.gsql",
              queryFileLocation / "xlnx_maximal_indep_set.gsql"]

for qf in queryFiles:
    with open(qf) as fh:
        print(f"installing queries in {qf}...")
        qStrRaw = fh.read()
        qStr = qStrRaw.replace('@graph', graphName)
        print(conn.gsql(qStr))

print("\n--------- All queries installed ----------")


--------- Installing Queries ----------
installing queries in /opt/xilinx/apps/graphanalytics/integration/Tigergraph-3.x/mis/0.2/examples/travelplan/query/build_edges.gsql...
Using graph 'xgraph_tigergraph'
Query build_edges could not be found.
The query build_edges has been added!
Start installing queries, about 1 minute ...
build_edges query: curl -X GET 'http://127.0.0.1:9000/query/xgraph_tigergraph/build_edges'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.

installing queries in /opt/xilinx/apps/graphanalytics/integration/Tigergraph-3.x/mis/0.2/examples/travelplan/query/tg_maximal_indep_set.gsql...
Query tg_maximal_indep_set could not be found.
The query tg_maximal_indep_set has been added!
Start installing queries, about 1 minute ...
tg_maximal_indep_set query: curl -X GET 'http://127.0.0.1:9000/query/xgraph_tigergraph/tg_maximal_indep_set?v_type=VALUE&e_type=VALUE&[max_iter=VALUE]&[print_accum=VALUE]&[file_path=VALUE]'. Add -H "Authorization: Bearer TOKEN" 

Now that queries are installed, rest of the operations can be performed simply by running the queries as follows.

### 2. Build edges <a id="build_edges"></a>
---

In [27]:
print('Building edges for travelplan vertices...')
tStart = time.perf_counter()
conn.runInstalledQuery('build_edges', timeout=240000000)
conn.runInstalledQuery('assign_ids', timeout=240000000)
conn.runInstalledQuery('build_csr', timeout=240000000)
print(f'completed in {time.perf_counter() - tStart:.4f} sec')

Building edges for travelplan vertices...
completed in 0.0773 sec


This completes the TigerGraph database preparation for MIS runs. We can now run as many MIS queries as we want. 

### Compute MIS <a id="run"></a>
---


In [33]:
print('Running Queries on FPGA...')
tStart = time.perf_counter()
result = conn.runInstalledQuery('maximal_indep_set_alveo', timeout=240000000)
tDuration = 1000*(time.perf_counter() - tStart)

for res in result:
    for k in res:
        print(k, ":", res[k])
    
print(f"\nRound Trip time: {tDuration:.2f} msec")

Running Queries on FPGA...
ExecTimeInMs : 223.18157
ComputationTechnique : Xilinx Alveo device
PeakVirtualMemoryInGB : 2.65854
PeakResidentMemoryInGB : 0.19234
MisSize : 414

Round Trip time: 234.01 msec


Feel free to play with the query!

#### Thanks for your time!